In [19]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
import google.generativeai as genai
from langgraph.checkpoint.memory import InMemorySaver
import os

# Load env variables
load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))

# Initialize model
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Define state
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

# Node 1: Generate Joke
def generate_joke(state: JokeState):
    prompt = f"Generate a joke on the topic: {state['topic']}"
    response = model.generate_content(prompt)
    return {"joke": response.text}   # Extract text

# Node 2: Generate Explanation
def generate_explain(state: JokeState):
    prompt = f"Write an explanation for the joke: {state['joke']}"
    response = model.generate_content(prompt)
    return {"explanation": response.text}   # Match with TypedDict key

# Build graph
graph = StateGraph(JokeState)
graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explain", generate_explain)

graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explain")
graph.add_edge("generate_explain", END)

# Compile workflow
checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

# Run
config1 = {"configurable": {"thread_id": "1"}}
result = workflow.invoke({"topic": "pizza"}, config=config1)

print(result)


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 54
}
]

In [21]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b98-682c-8000-b9c1d1894efe'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-15T16:32:01.491153+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b8e-6a7c-bfff-ea63b9cfe54d'}}, tasks=(PregelTask(id='d44d1f63-decf-8a9c-f8ee-8f6e6f4bbb27', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error="ResourceExhausted('You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.')", interrupts=(), state=None, result=None),), interrupts=())

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b98-682c-8000-b9c1d1894efe'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-15T16:32:01.491153+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b8e-6a7c-bfff-ea63b9cfe54d'}}, tasks=(PregelTask(id='d44d1f63-decf-8a9c-f8ee-8f6e6f4bbb27', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error="ResourceExhausted('You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.')", interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={}, next=('__start__',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b8e-6a7c-bfff-ea63b9cfe54d'}}, metadata={'source': 'i

In [23]:
config2={'configurable': {"thread_id":"2"}}
workflow.invoke({"topic":'pasta'}, config=config2)

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 7
}
]

In [24]:
#time travel

workflow.get_state({'configurable':{'thread_id':"1","checkpoint_id":"1f092518-1b8e-6a7c-bfff-ea63b9cfe54d"}})

StateSnapshot(values={}, next=('__start__',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f092518-1b8e-6a7c-bfff-ea63b9cfe54d'}}, metadata={'source': 'input', 'step': -1, 'parents': {}}, created_at='2025-09-15T16:32:01.487116+00:00', parent_config=None, tasks=(PregelTask(id='1d08400b-b4d4-7972-9165-4290d49c3cb2', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'topic': 'pizza'}),), interrupts=())

In [25]:
workflow.invoke(None,{"configurable":{"thread_id":"1f092518-1b8e-6a7c-bfff-ea63b9cfe54d"}})

EmptyInputError: Received no input for __start__

In [27]:
workflow.update_state({"configurable":{"thread_id":"1", "checkpoint_id":"1f092518-1b8e-6a7c-bfff-ea63b9cfe54d","checkpoint_ns":""}},{'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f092556-834e-6386-8000-62f45422671d'}}

In [28]:

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092556-834e-6386-8000-62f45422671d'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-09-15T16:59:56.665741+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b8e-6a7c-bfff-ea63b9cfe54d'}}, tasks=(PregelTask(id='35fb2e90-adf6-d482-0b49-c2f5ebb0e9b8', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f092518-1b98-682c-8000-b9c1d1894efe'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-15T16:32:01.491153+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': 

In [29]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})

EmptyInputError: Received no input for __start__